In [1]:
from cv2 import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import random

depth_images = ['./Dataset_Assignment_2/depth_0.jpg','./Dataset_Assignment_2/depth_1.jpg','./Dataset_Assignment_2/depth_2.jpg','./Dataset_Assignment_2/depth_3.jpg']

images = ['./Dataset_Assignment_2/im_0.jpg','./Dataset_Assignment_2/im_1.jpg','./Dataset_Assignment_2/im_2.jpg','./Dataset_Assignment_2/im_3.jpg']



In [30]:
im1 = images[0]
d1 = depth_images[0]
im2 = images[1]

image1 = cv2.imread(im1)
image2 = cv2.imread(im2)

depth_img1 = cv2.imread(d1)
#

def get_keypoints(image):
    sift = cv2.SIFT_create()
    keyp,des = sift.detectAndCompute(image,None)
    
    return keyp,des

# here we are using the Lowe's ratio test for matching
def get_matchings(des1,des2):
    bf_matcher = cv2.BFMatcher()
    matchings = bf_matcher.knnMatch(des1,des2,k=2)
    matchings_list = []
    for a,b in matchings:
        if(a.distance<0.7*b.distance):
            matchings_list.append([a])
    return matchings_list

# we will now use the above functions to calculate the matches
keyp1,des1 = get_keypoints(image1)
keyp2,des2 = get_keypoints(image2)

quality_matchings = get_matchings(des1,des2)


# Now we are dividing the all depths interval into 5 levels
# we will use the m = 52
levels = [[],[],[],[],[]]
# for inbuilt homography
levels_query = [[],[],[],[],[]]
levels_train = [[],[],[],[],[]]


for i in range(len(quality_matchings)):
    img_coordinates = list(keyp1[quality_matchings[i][0].queryIdx].pt)
    img_coordinates2 = list(keyp2[quality_matchings[i][0].trainIdx].pt)
    
    x_coordinate = img_coordinates[0]
    y_coordinate = img_coordinates[1]
    x_coordinate1 = img_coordinates2[0]
    y_coordinate1 = img_coordinates2[1]
    
    
    depth = depth_img1[int(y_coordinate),int(x_coordinate)][0]
    # now we have inserted a particular point in accordance with its depth
    levels[int(np.floor(depth/52))].append([x_coordinate,y_coordinate,x_coordinate1,y_coordinate1])
    
    levels_query[int(np.floor(depth/52))].append(keyp1[quality_matchings[i][0].queryIdx].pt)
    levels_train[int(np.floor(depth/52))].append(keyp2[quality_matchings[i][0].trainIdx].pt)
    
    

# Here we are ensuring every one has got enough points else we are replacing it with its neighboour
for i in range(len(levels)):
    if(len(levels[i]) < 20):
        flag = 0
        for j in range(i+1,5):
            if(len(levels[j])>20):
                levels[i] = levels[j]
                levels_query[i] = levels_query[j]
                levels_train[i] = levels_train[j]
                flag = 1
                break
        if(flag != 1):
            for j in range(i-1,-1,-1):
                if(len(levels[j])>20):
                    levels[i] = levels[j]
                    levels_query[i] = levels_query[j]
                    levels_train[i] = levels_train[j]
                    break




    
    

In [31]:
def find_homography_matrix(random_points):
    #looping through the four random points and finding the Assemble matrix
    matra = []
    for point in random_points:
        point1 = np.matrix([point.item(0), point.item(1), 1])
        point2 = np.matrix([point.item(2), point.item(3), 1])

        matra2 = [0, 0, 0, -point2.item(2) * point1.item(0), -point2.item(2) * point1.item(1), -point2.item(2) * point1.item(2),
              point2.item(1) * point1.item(0), point2.item(1) * point1.item(1), point2.item(1) * point1.item(2)]

        matra1 = [-point2.item(2) * point1.item(0), -point2.item(2) * point1.item(1), -point2.item(2) * point1.item(2), 0, 0, 0,
              point2.item(0) * point1.item(0), point2.item(0) * point1.item(1), point2.item(0) * point1.item(2)]
        
        matra.append(matra1)
        matra.append(matra2)

    Assemble_matrix = np.matrix(matra)

    #dividing assemble matri into the svd
    u, s, v = np.linalg.svd(Assemble_matrix)

    #reshape the min singular value into a 3 by 3 matrix
    h = np.reshape(v[8], (3, 3))
    
    #normalize and now we have h
    h = (1/h.item(8)) * h
    return h


#### error between estimated and real #####
def Distance_for_point(point, Homography_temp):

    point1 = np.transpose(np.matrix([point[0].item(0), point[0].item(1), 1]))
    estimate_point2 = np.dot(Homography_temp, point1)
    estimate_point2 = (1/estimate_point2.item(2))*estimate_point2

    point2 = np.transpose(np.matrix([point[0].item(2), point[0].item(3), 1]))
    error = point2 - estimate_point2
    return np.linalg.norm(error)


def ransac(matches,threshold):
    maximum_inliers = []
    Homography_matrix = []
    for i in range(1000):
        #taking 4 random points for calculating homography matrix
        a = matches[random.randrange(0, len(matches))]
        b = matches[random.randrange(0, len(matches))]
        
        c = matches[random.randrange(0, len(matches))]
        
        d = matches[random.randrange(0, len(matches))]
        
        four_points = np.vstack((a, b))
        four_points = np.vstack((four_points, c))
        four_points = np.vstack((four_points, d))

        #call the homography function on those points
        Homography_temp = find_homography_matrix(four_points)
        inliers = []

        for i in range(len(matches)):
            d = Distance_for_point(matches[i], Homography_temp)
            if d < threshold:
                inliers.append(matches[i])
        

        if len(inliers) > len(maximum_inliers):
            maximum_inliers = inliers
            Homography_matrix = Homography_temp
    

    return Homography_matrix, maximum_inliers




H_inbuilt = []
H_ours = []

for i in range(len(levels)):
    if(len(levels[i])>0):
        abc = np.matrix(levels[i])
        Homographymtr,inliers = ransac(abc,5)
        
        qr = np.matrix(levels_query[i])
        tr = np.matrix(levels_train[i])
       
        Homographyinbuilt,status = cv2.findHomography(qr,tr,cv2.RANSAC)
        H_ours.append(Homographymtr)
        H_inbuilt.append(Homographyinbuilt)

        

    

Homography: [[ 1.57590056e+00  8.69657693e-03 -2.20966958e+02]
 [ 1.77022011e-01  1.36628692e+00 -1.00853601e+02]
 [ 8.67708540e-04  1.64038582e-04  1.00000000e+00]]
Homography inbuilt: [[ 1.63925907e+00  2.43313538e-02 -2.35547522e+02]
 [ 1.84765109e-01  1.44018442e+00 -1.12587828e+02]
 [ 9.05946409e-04  2.82065287e-04  1.00000000e+00]]
0
Homography: [[ 1.62300748e+00  3.68203456e-03 -2.29897895e+02]
 [ 1.93237907e-01  1.41889598e+00 -1.13022785e+02]
 [ 9.26469070e-04  1.99646346e-04  1.00000000e+00]]
Homography inbuilt: [[ 1.54081562e+00 -1.60680035e-02 -2.12206079e+02]
 [ 1.83261911e-01  1.30908529e+00 -9.50670184e+01]
 [ 8.81073043e-04  2.59461835e-05  1.00000000e+00]]
1
Homography: [[ 1.45407767e+00 -5.74154118e-02 -1.87244958e+02]
 [ 1.57785660e-01  1.21189208e+00 -7.35426880e+01]
 [ 7.78653891e-04 -6.05012037e-05  1.00000000e+00]]
Homography inbuilt: [[ 1.52994919e+00 -2.30943943e-02 -2.08948418e+02]
 [ 1.70009240e-01  1.31995243e+00 -9.24824664e+01]
 [ 8.36092068e-04  8.6394013